In [1]:
from metabolabpytools import isotopomerAnalysis
import re

def sanitize_sheet_name(name):
    return re.sub(r'[:\\/*?\[\]]', '_', name)

# Define the HSQC vectors
hsqc_vectors = [
    [1, 1, 1],
    [0, 1, 1],
    [0, 0, 1],
    [0, 1, 1, 0],
    [1, 1, 1, 0],
    [0, 1, 1, 1],
    [1, 1, 1, 1],
    [0, 1, 1, 1, 0],
    [1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]

# Loop through each HSQC vector and simulate data
for hsqc_vector in hsqc_vectors:
    num_carbons = len(hsqc_vector)
    metabolite = f'{num_carbons}C_HSQC_{hsqc_vector}'
    
    # Sanitize the sheet name
    sanitized_metabolite = sanitize_sheet_name(metabolite)
    
    # Initialize the analysis class and metabolite
    ia = isotopomerAnalysis.IsotopomerAnalysis()
    ia.init_metabolite(sanitized_metabolite, hsqc_vector)
    
    # Simulate data
    hsqc_data, gcms_data, isotopomer_distributions = ia.simulate_and_save_data(
        num_samples=1000,
        num_carbons=num_carbons,
        metabolite=sanitized_metabolite
    )




Simulated data for 3C_HSQC__1, 1, 1_
HSQC Data Shape: (1000, 8)
GC-MS Data Shape: (1000, 4)
Isotopomer Distributions Shape: (1000, 8)
Simulated data for 3C_HSQC__0, 1, 1_
HSQC Data Shape: (1000, 8)
GC-MS Data Shape: (1000, 4)
Isotopomer Distributions Shape: (1000, 8)
Simulated data for 3C_HSQC__0, 0, 1_
HSQC Data Shape: (1000, 8)
GC-MS Data Shape: (1000, 4)
Isotopomer Distributions Shape: (1000, 8)
Simulated data for 4C_HSQC__0, 1, 1, 0_
HSQC Data Shape: (1000, 12)
GC-MS Data Shape: (1000, 5)
Isotopomer Distributions Shape: (1000, 16)
Simulated data for 4C_HSQC__1, 1, 1, 0_
HSQC Data Shape: (1000, 12)
GC-MS Data Shape: (1000, 5)
Isotopomer Distributions Shape: (1000, 16)
Simulated data for 4C_HSQC__0, 1, 1, 1_
HSQC Data Shape: (1000, 12)
GC-MS Data Shape: (1000, 5)
Isotopomer Distributions Shape: (1000, 16)
Simulated data for 4C_HSQC__1, 1, 1, 1_
HSQC Data Shape: (1000, 12)
GC-MS Data Shape: (1000, 5)
Isotopomer Distributions Shape: (1000, 16)
Simulated data for 5C_HSQC__0, 1, 1, 1, 0_

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import joblib

def flatten_hsqc_multiplets(hsqc_multiplet_percentages):
    return [item for sublist in hsqc_multiplet_percentages for item in (sublist if isinstance(sublist, list) else [sublist])]

def add_noise_to_hsqc(hsqc_data, noise_level=0.03):
    noise = np.random.normal(0, noise_level, hsqc_data.shape)
    noisy_hsqc_data = hsqc_data * (1 + noise)
    return noisy_hsqc_data

def add_noise_to_gcms(gcms_data, noise_level=0.075):
    noise = np.random.normal(0, noise_level, gcms_data.shape)
    noisy_gcms_data = gcms_data * (1 + noise)
    return noisy_gcms_data

def prepare_data(hsqc_data, gcms_data, isotopomer_distributions, test_size=0.2):
    noisy_hsqc_data = add_noise_to_hsqc(hsqc_data)
    noisy_gcms_data = add_noise_to_gcms(gcms_data)
    
    X = np.hstack((noisy_hsqc_data, noisy_gcms_data))
    y = isotopomer_distributions
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return X_train, X_test, y_train, y_test

def build_and_train_model(X_train, y_train, X_val, y_val, input_dim, epochs=100, batch_size=32):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='relu'))
    
    model.compile(optimizer=Adam(), loss='mean_absolute_error')
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])
    
    return model

def normalize_predictions(predictions):
    normalized_predictions = np.maximum(predictions, 0)
    normalized_predictions = normalized_predictions / np.sum(normalized_predictions, axis=1, keepdims=True) * 100
    return normalized_predictions

def monte_carlo_simulation(model, X_test, y_test, num_simulations=1000):
    predictions = []
    
    for _ in range(num_simulations):
        noisy_X_test = add_noise_to_hsqc(X_test[:, :X_test.shape[1]//2])
        noisy_X_test = np.hstack((noisy_X_test, add_noise_to_gcms(X_test[:, X_test.shape[1]//2:])))
        
        preds = model.predict(noisy_X_test)
        preds = normalize_predictions(preds)
        predictions.append(preds)
    
    predictions = np.array(predictions)
    mean_preds = np.mean(predictions, axis=0)
    std_preds = np.std(predictions, axis=0)
    
    return mean_preds, std_preds

def save_model(model, filename='best_model.pkl'):
    joblib.dump(model, filename)

def generate_results_sheet(X_test, y_test, model, mean_preds, std_preds, mae, filename='results.xlsx'):
    import pandas as pd
    from openpyxl import Workbook
    
    predictions = model.predict(X_test)
    predictions = normalize_predictions(predictions)
    
    wb = Workbook()
    sheet = wb.active
    sheet.title = 'Results'
    sheet.append(["Sample", "Real Isotopomer Distribution", "HSQC %", "GC-MS %", "Predicted Isotopomer Distribution", "MAE", "Std Dev"])
    
    for i in range(X_test.shape[0]):
        sample = i + 1
        real_isotopomer_dist = y_test[i].tolist()
        hsqc_percentage = X_test[i][:X_test.shape[1]//2].tolist()
        gcms_percentage = X_test[i][X_test.shape[1]//2:].tolist()
        predicted_isotopomer_dist = predictions[i].tolist()
        std_dev = std_preds[i].tolist()
        
        real_isotopomer_dist_str = str(real_isotopomer_dist)
        hsqc_percentage_str = str(hsqc_percentage)
        gcms_percentage_str = str(gcms_percentage)
        predicted_isotopomer_dist_str = str(predicted_isotopomer_dist)
        std_dev_str = str(std_dev)
        
        sheet.append([sample, real_isotopomer_dist_str, hsqc_percentage_str, gcms_percentage_str, predicted_isotopomer_dist_str, mae, std_dev_str])
    
    wb.save(filename)

# HSQC vectors to loop through
hsqc_vectors = [
    [1, 1, 1], [0, 1, 1], [0, 0, 1],
    [0, 1, 1, 0], [1, 1, 1, 0], [0, 1, 1, 1], [1, 1, 1, 1],
    [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]

# Create a directory for storing results
results_dir = "results_sim"
os.makedirs(results_dir, exist_ok=True)

# Loop through each HSQC vector
for hsqc_vector in hsqc_vectors:
    # Generate the file path based on the HSQC vector
    vector_str = ", ".join(map(str, hsqc_vector))
    num_carbons = len(hsqc_vector)
    file_path = f'data_sim/{num_carbons}C_HSQC__{vector_str}__simulation_results.xlsx'
    
    df = pd.read_excel(file_path)
    
    X_hsqc = df['HSQC Multiplet %'].apply(eval).apply(flatten_hsqc_multiplets).tolist()
    X_gcms = df['GC-MS %'].apply(eval).tolist()
    X = np.array([flatten_hsqc_multiplets(hsqc) + gcms for hsqc, gcms in zip(X_hsqc, X_gcms)])
    
    y = df['Isotopomer Distribution'].apply(eval).tolist()
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = prepare_data(X[:, :len(X_hsqc[0])], X[:, len(X_hsqc[0]):], y, test_size=0.2)
    
    input_dim = X_train.shape[1]
    model = build_and_train_model(X_train, y_train, X_test, y_test, input_dim)
    
    mean_preds, std_preds = monte_carlo_simulation(model, X_test, y_test)
    
    mae = model.evaluate(X_test, y_test, verbose=0)
    
    model_filename = os.path.join(results_dir, f'{num_carbons}C_HSQC__{vector_str}__best_model.pkl')
    save_model(model, filename=model_filename)
    
    results_filename = os.path.join(results_dir, f'{num_carbons}C_HSQC__{vector_str}__results.xlsx')
    generate_results_sheet(X_test, y_test, model, mean_preds, std_preds, mae, filename=results_filename)


Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 10.2716 - val_loss: 6.8535
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.7347 - val_loss: 5.4008
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1614 - val_loss: 4.1312
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9042 - val_loss: 3.4088
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.4353 - val_loss: 3.1653
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1526 - val_loss: 2.9973
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0243 - val_loss: 2.8944
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9512 - val_loss: 2.8400
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.8757 - val_loss: 2.8016
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6773 - val_loss: 2.5634
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5464 - val_loss: 2.5355
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.

In [36]:
import os
import pandas as pd
import numpy as np
from keras.models import load_model
from openpyxl import Workbook
import joblib

# Helper function to flatten HSQC multiplets
def flatten_hsqc_multiplets(hsqc_multiplet_percentages):
    return [item for sublist in hsqc_multiplet_percentages for item in (sublist if isinstance(sublist, list) else [sublist])]

# Helper function to normalize predictions
def normalize_predictions(predictions):
    normalized_predictions = np.maximum(predictions, 0)
    normalized_predictions = normalized_predictions / np.sum(normalized_predictions, axis=1, keepdims=True) * 100
    return normalized_predictions

# Helper function to add noise to HSQC data
def add_noise_to_hsqc(hsqc_data, noise_level=0.03):
    noise = np.random.normal(0, noise_level, hsqc_data.shape)
    noisy_hsqc_data = hsqc_data * (1 + noise)
    return noisy_hsqc_data

# Helper function to add noise to GC-MS data
def add_noise_to_gcms(gcms_data, noise_level=0.075):
    noise = np.random.normal(0, noise_level, gcms_data.shape)
    noisy_gcms_data = gcms_data * (1 + noise)
    return noisy_gcms_data

# Function to prepare the data for prediction
def prepare_data(hsqc_data, gcms_data):
    # Combine HSQC and GC-MS data
    X = np.hstack((hsqc_data, gcms_data))
    return X

# Function to perform Monte Carlo simulation
def monte_carlo_simulation(model, X_test, num_simulations=1000):
    predictions = []
    
    for _ in range(num_simulations):
        noisy_X_test = add_noise_to_hsqc(X_test[:, :X_test.shape[1]//2])
        noisy_X_test = np.hstack((noisy_X_test, add_noise_to_gcms(X_test[:, X_test.shape[1]//2:])))
        
        preds = model.predict(noisy_X_test)
        preds = normalize_predictions(preds)
        predictions.append(preds)
    
    predictions = np.array(predictions)
    mean_preds = np.mean(predictions, axis=0)
    std_preds = np.std(predictions, axis=0)
    
    return mean_preds, std_preds

# Function to generate the results sheet
def generate_results_sheet(X_test, y_test, model, mean_preds, std_preds, mae, filename='results.xlsx'):
    predictions = model.predict(X_test)
    predictions = normalize_predictions(predictions)
    
    wb = Workbook()
    sheet = wb.active
    sheet.title = 'Results'
    sheet.append(["Sample", "Real Isotopomer Distribution", "HSQC %", "GC-MS %", "Predicted Isotopomer Distribution", "MAE", "Std Dev"])
    
    for i in range(X_test.shape[0]):
        sample = i + 1
        real_isotopomer_dist = y_test[i].tolist() if y_test is not None else 'N/A'
        hsqc_percentage = X_test[i][:X_test.shape[1]//2].tolist()
        gcms_percentage = X_test[i][X_test.shape[1]//2:].tolist()
        predicted_isotopomer_dist = predictions[i].tolist()
        std_dev = std_preds[i].tolist()
        
        real_isotopomer_dist_str = str(real_isotopomer_dist)
        hsqc_percentage_str = str(hsqc_percentage)
        gcms_percentage_str = str(gcms_percentage)
        predicted_isotopomer_dist_str = str(predicted_isotopomer_dist)
        std_dev_str = str(std_dev)
        
        sheet.append([sample, real_isotopomer_dist_str, hsqc_percentage_str, gcms_percentage_str, predicted_isotopomer_dist_str, mae, std_dev_str])
    
    wb.save(filename)

# Define the directory for reading experimental data and loading models
experimental_data_dir = 'experimental_data'
results_dir = 'experimental_results'
os.makedirs(results_dir, exist_ok=True)

# Load HSQC data
hsqc_df = pd.read_excel('C:/Users/raath/metabolabpytools/metabolabpytools/hsqcData1.xlsx', sheet_name=None)

# Load GC-MS data
gcms_df = pd.read_excel('C:/Users/raath/metabolabpytools/metabolabpytools/gcmsData1.xlsx', sheet_name=None)

# Function to assign HSQC percentages to correct multiplets
def assign_hsqc_percentages(multiplets, percentages, num_carbons):
    all_multiplets = {
        3: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2]],
        4: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5]],
        5: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5], [5], [5, 4], [5, 6], [5, 4, 6]],
        6: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5], [5], [5, 4], [5, 6], [5, 4, 6], [6], [6, 5], [6, 7], [6, 5, 7]]
    }
    print(all_multiplets)
    assigned_percentages = []
    multiplets = [eval(m) if isinstance(m, str) else m for m in multiplets]
    multiplets = [m if isinstance(m, tuple) else (m,) for m in multiplets]
    for multiplet in all_multiplets[num_carbons]:
        if tuple(multiplet) in multiplets:
            idx = multiplets.index(tuple(multiplet))
            assigned_percentages.append(percentages[idx])
        else:
            assigned_percentages.append(0)
    return assigned_percentages

# Loop through the HSQC sheets
for sheet_name, hsqc_data in hsqc_df.items():
    if sheet_name == "PyruvicAcid(Enol)":
        continue  # Skip the special case for now

    # Extract GC-MS data for the same sheet
    gcms_data = gcms_df.get(sheet_name)
    
    if gcms_data is None:
        print(f"GC-MS data for {sheet_name} not found, skipping...")
        continue
    
    # Prepare HSQC and GC-MS data
    multiplets = hsqc_data['Multiplet.0'].tolist()
    print(multiplets)
    percentages = hsqc_data['Percentages.0'].tolist()
    print(percentages)
    num_carbons = len(hsqc_data['HSQC.0'].iloc[0].replace(' ', ''))
    print(num_carbons)
    
    hsqc_percentages = [assign_hsqc_percentages(multiplets, percentages, num_carbons) for _ in range(len(multiplets))]
    
    # Flatten HSQC data
    X_hsqc_assigned = np.array([flatten_hsqc_multiplets(hsqc) for hsqc in hsqc_percentages])
    X = np.hstack((X_hsqc_assigned, gcms_data['Percentages.0'].tolist()))
    
    # Get HSQC vector and number of carbons
    hsqc_vector = hsqc_data['HSQC.0'].iloc[0]


['2', '2, 1', '2, 3', '2, 1, 3', '3', '3, 2', '3, 4', '3, 2, 4', '3', '3, 2', '3, 4', '3, 2, 4', '4', '4, 3', '4, 5', '4, 3, 5']
[40.832, 45.408, 13.761, 0.0, 64.315, 7.68, 25.759, 2.246, 65.249, 8.075, 24.87, 1.806, 19.696, 2.213, 72.824, 5.267]
5
{3: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2]], 4: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5]], 5: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5], [5], [5, 4], [5, 6], [5, 4, 6]], 6: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5], [5], [5, 4], [5, 6], [5, 4, 6], [6], [6, 5], [6, 7], [6, 5, 7]]}
{3: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2]], 4: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5]], 5: [[2], [2, 1], [2, 3], [2, 1, 3], [3], [3, 2], [3, 4], [3, 2, 4], [4], [4, 3], [4, 5], [4, 3, 5], [5], [5, 4], [5, 6], [5, 4, 6]], 6: [[2], [2, 1],

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
num_carbons = len(hsqc_data['HSQC.0'].iloc[0].replace(' ', ''))